In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import rasterio
from rasterio import features
import shapely
from shapely.geometry import Point, Polygon

df = pd.read_csv("../input/hubmap-organ-segmentation/train.csv")
df['image_path'] = '../input/hubmap-organ-segmentation/train_images/'
df['image_path'] = df['image_path'].str.cat(df['id'].astype(str))
df['image_path'] = df['image_path'] + '.tiff'
df.head()

In [ ]:
def helper(mask, img_shape):
  
    canvas = np.zeros(img_shape).T
    canvas[tuple(zip(*mask))] = 1.0

      # This is the Equivalent for loop of the above command for better understanding.
      # for pos in range(len(p_loc)):
      #   canvas[pos[0], pos[1]] = 1

    return canvas

def get_mask(rle_string, img_shape):
    rle = [int(i) for i in rle_string.split(' ')]
    pairs = list(zip(rle[0::2],rle[1::2]))

    p_loc = []

    for start, length in pairs:
        for p_pos in range(start, start + length):
            p_loc.append((p_pos % img_shape[1], p_pos // img_shape[0]))
  
    return helper(p_loc, img_shape)


def mask_to_polygons_layer(mask:np.array) -> shapely.geometry.Polygon:
    """Converting mask to polygon object
    
    Input:
        mask: (np.array): Image like Mask [0,1] where all 1 are consider as masks
        
    Output:
        shapely.geometry.Polygon: Polygons
    
    """
    all_polygons = []
    for shape, value in features.shapes(mask.astype(np.int16), mask=(mask >0), transform=rasterio.Affine(1.0, 0, 0, 0, 1.0, 0)):
        all_polygons.append(shapely.geometry.shape(shape))

    all_polygons = shapely.geometry.MultiPolygon(all_polygons)
    
    if not all_polygons.is_valid:
        all_polygons = all_polygons.buffer(0)
        # Sometimes buffer() converts a simple Multipolygon to just a Polygon,
        # need to keep it a Multi throughout
        if all_polygons.type == 'Polygon':
            all_polygons = shapely.geometry.MultiPolygon([all_polygons])
    return all_polygons

def get_date(id:int) -> tuple:
    """Get Important data
    
    Input:
        id: (int) id of the data
        get_vals: (list) List of the cols you want to get from the data
    
    Output:
        tuple:
            int: Image id
            tuple: Image shape (height, width)
            image_path: Path of the image
            image: PIL Image object 
            polygons: Polygon points list
            mask: Mask of the image
        
    """
    get_vals = ["id",'img_height', 'img_width','image_path', 'rle']
    values = list()
    for i in get_vals:
        values.append(list(df[df["id"]==id][i].values)[0])

    # Getting image data
    img_id, img_height, img_width, image_path, rle = values
    img_shape = (img_width, img_height)
    image = Image.open(image_path)
    
    mask = get_mask(rle, img_shape)
    
    # Converting Mask to Polygon
    polygons = mask_to_polygons_layer(mask)
    polygons = [list(poly.exterior.coords) for poly in list(polygons)]
    return (img_id, img_shape, image_path, image, polygons, mask)



# Draw Polygon

In [ ]:
import math
from PIL import Image, ImageDraw
from PIL import ImagePath 
  
for part in df['organ'].unique():
    
    idx = list(df[df['organ']==part]["id"].values)[0]
    
    img_id, img_shape, image_path, image, polygons, mask = get_date(idx)
    img1 = ImageDraw.Draw(image) 

    for points in polygons:
        img1.polygon(points,fill="#eeeeee", outline ="blue", width=9) 
    
    
    plt.figure(figsize=(20,10))
    plt.imshow(image)
    plt.title(part)
    plt.show()

# Draw Masks

In [ ]:
for part in df['organ'].unique():

  x = df[df['organ'] == part]
  index_list = x.index
  idx = index_list[np.random.randint(0, x.shape[0])]

  class_of_scan = df.loc[idx, 'organ']
  image_path = df.loc[idx, 'image_path']
  id = df.loc[idx, 'id'] 
  
  image = np.array(Image.open(image_path)) / 255
  k = (df.loc[idx, 'img_height'], df.loc[idx, 'img_width'])

  rle_string = df.loc[idx, 'rle']
  mask = get_mask(rle_string, k)


  fig, ax = plt.subplots(1,3, figsize=(15,15))
  ax[0].set_title(f'Image : {id}')
  ax[0].imshow(image)

  ax[1].set_title(f'Mask : {id}')
  ax[1].imshow(mask)

  ax[2].set_title(f'{class_of_scan} Segmented : {id}')
  ax[2].imshow(np.dstack((mask, np.zeros(mask.shape), np.argmax(image, axis=-1))))
  plt.show()